In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import norm
from scipy.spatial.distance import euclidean
import random
import matplotlib.pyplot as plt
from numpy import genfromtxt
from scipy.spatial.distance import pdist, squareform
import pickle
from sklearn.cluster import KMeans
import umap.umap_ as umap
import os

In [21]:
#text contents
text = ["norm_reads_matrix" + str(num1)+".csv"for num1  in range(1,32)]
text2=["gene_name_text"+str(num1)+".csv"for num1  in range(1,32)]
text3=["pkl"+str(num1)for num1  in range(1,32)]
text4=["SSE"+str(num1)+".png"for num1  in range(1,32)]
text5=["K-means"+str(num1)+".png"for num1  in range(1,32)]
text6=["pkl"+str(num1)for num1 in range(1,32)]
text7=['scatterplot'+str(num1)+".png"for num1 in range(1,32)]
text8=['3cluster_sample'+str(num1)+".png"for num1 in range(1,32)]

In [18]:
#test for one sample
for k in range(0,31):
                df=np.genfromtxt(text[k],delimiter=",",skip_header=True)[:,1:]
                for i in range(0,df.shape[0]):
                      df[i,]=df[i,:]/df[i,:].sum()
                out = squareform(pdist(np.sqrt(df)))/np.sqrt(2)
                umap1=umap.UMAP(n_neighbors=15, 
                            min_dist=0.1, 
                            n_components=2, 
                            metric='euclidean',
                            random_state=42).fit_transform(out)
                
                #SSE estimate best K value
                kmeans_kwargs = {"init": "random","n_init": 10,"max_iter": 300,"random_state": 42}
                from sklearn.cluster import KMeans
                sse = []
                for j in range(1, 11):
                        kmeans = KMeans(n_clusters=j)
                        kmeans.fit(umap1)
                        sse.append(kmeans.inertia_)
                plt.style.use("fivethirtyeight")
                plt.plot(range(1, 11), sse)
                plt.xticks(range(1, 11))
                plt.xlabel("Number of Clusters")
                plt.ylabel("SSE")
                plt.title("Elbow plot of SSE")
                plt.savefig("SSE/"+text4[k],format="png",bbox_inches='tight',dpi=600)
                
                #try k equals 3...
                v=3
                kmeans = KMeans(n_clusters=v)
                result=kmeans.fit(umap1)
                labels1=result.labels_

                #visulize clustering result
                df=pd.DataFrame(df)
                umap1=pd.DataFrame(umap1)
                labels1=pd.DataFrame(labels1)+1
                final=pd.concat([umap1,labels1],axis=1)
                final.columns=['umap1','umap2','labels']
                fin_res=pd.concat([final,df],axis=1)

                #relabel by location
                rank1=np.array([1,2,3])
                rank2=np.array(fin_res.groupby('labels')['umap1'].mean().rank())
                fin_res['labels'] = fin_res['labels'].replace(rank1,rank2)
                gene_name=pd.read_csv(text2[k])
                final=pd.concat([gene_name,labels1],axis=1)
                with open(text6[k],'wb') as f:
                             pickle.dump(final, f)

                #sample the data
                group_sample=[]
                for s in range(1,4):
                             group=fin_res[fin_res.iloc[:,2]==s]
                             tmp=group.sample(n=10)
                             group_sample.append(tmp)
                group_sample=pd.concat(group_sample)
                group_sample.sort_values(by='labels',ascending=True,inplace=True)

                #plot all 30 dots in the pics
                plt.figure(figsize=(10,10))
                sns.scatterplot(x=fin_res['umap1'],y=fin_res['umap2'],hue=fin_res['labels'],palette=['red','blue','green'])
                plt.savefig("3cluster/"+text7[k],format="png",bbox_inches='tight',dpi=600)

                #plot three cluster 
                plt.figure(figsize=(30, 30))
                plt.subplots_adjust(wspace=0.5, hspace=0.7)
                for d in range(1,10):
                                 plt.subplot(10,3,(3*d-2))
                                 x=range(-1000,1001)
                                 plt.ylim((0, 0.008))
                                 y=group_sample.iloc[(d-1),3:]
                                 plt.plot(x,y,color="red")
                for d in range(11,20):
                                 plt.subplot(10,3,(3*d-31))
                                 x=range(-1000,1001)
                                 plt.ylim((0, 0.008))
                                 y=group_sample.iloc[(d-1),3:]
                                 plt.plot(x,y,color="blue")
                for d in range(21,30):
                                 plt.subplot(10,3,(3*d-60))
                                 x=range(-1000,1001)
                                 plt.ylim((0, 0.008))
                                 y=group_sample.iloc[(d-1),3:]
                                 plt.plot(x,y,color="green")
                d=10
                plt.subplot(10,3,(3*d-2))
                x=range(-1000,1001)
                plt.ylim((0, 0.008))
                y=group_sample.iloc[(d-1),3:]
                plt.plot(x,y,color="red")
                plt.title('cluster1', y=-1)

                d=20
                plt.subplot(10,3,(3*d-31))
                x=range(-1000,1001)
                plt.ylim((0, 0.01))
                y=group_sample.iloc[(d-1),3:]
                plt.plot(x,y,color="blue")
                plt.title('cluster2',y=-1)

                d=30
                plt.subplot(10,3,(3*d-60))
                x=range(-1000,1001)
                plt.ylim((0, 0.008))
                y=group_sample.iloc[(d-1),3:]
                plt.plot(x,y,color="green")
                plt.title('cluster3',y=-1)
                plt.savefig(text8[k],format="png",bbox_inches='tight',dpi=300)
                plt.show()
                print(k)